# Analyse Commit Classification 

Analysis of Prospector reports with respect to commit classification. Compares results with and without commit classification.

In [2]:
import json
import sys
import os
from omegaconf import OmegaConf

# print(os.getcwdb())
config = OmegaConf.load("../config.yaml")

## Analysis

### How many fixing commits have been correctly obtained by Prospector? 

Find out how many fixing commits have been obtained correctly using Prospector: 

In [5]:
def count_correct_results(ground_truth_data, prospector_data):
    count_correct = 0
    count_correct_first_3 = 0

    for cve_record in ground_truth_data["ground_truth"]:
        cve_id = cve_record["vulnerability_id"]

        prospector_cve_record = prospector_data["vulnerabilities"].get(cve_id, None)
        if prospector_cve_record is not None:
            # print(f"{cve_id} in both baseline and prospector.") # Sanity check

            true_fixing_commits = [
                commit["id"] for fix in cve_record["fixes"] for commit in fix["commits"]
            ]

            if prospector_cve_record["commits"]:
                first_fixing_commit = prospector_cve_record["commits"][0]["commit_hash"]
                first_3_fixing_commits = [
                    commit["commit_hash"]
                    for commit in prospector_cve_record["commits"][:3]
                ]

            if first_fixing_commit in true_fixing_commits:
                count_correct += 1

            if set(first_3_fixing_commits) & set(true_fixing_commits):
                count_correct_first_3 += 1

    return count_correct, count_correct_first_3

In [6]:
with open("../../../" + config.ground_truth, "r") as f1, open(
    "../../../" + config.no_cc, "r"
) as f2:
    ground_truth_data = json.load(f1)  # baseline from project KB
    prospector_results = json.load(f2)  # prospector results without cc

cves_2018 = []
for file in os.listdir("../../../data_sources/reports/"):
    if file.startswith("CVE-2018-"):
        cves_2018.append(file)

correct_is_top, correct_in_top_3 = count_correct_results(
    ground_truth_data, prospector_results
)
print(
    f"{correct_is_top} times the top candidate was the true fixing commit, \n{correct_in_top_3} times the top 3 candidates contained the true fixing commit out of {len(cves_2018)} reports."
)

91 times the top candidate was the true fixing commit, 
112 times the top 3 candidates contained the true fixing commit out of 18 reports.


##### 2018 CVEs

Without Commit Classification Rule: 

In [ ]:
with open("../../../" + config.ground_truth, "r") as f1, open(
    "../../../" + config.no_cc, "r"
) as f2:
    ground_truth_data = json.load(f1)  # baseline from project KB
    prospector_results = json.load(f2)  # prospector results without cc

cves_2018 = []
for file in os.listdir("../../../data_sources/reports/"):
    if file.startswith("CVE-2018-"):
        cves_2018.append(file)

correct_is_top, correct_in_top_3 = count_correct_results(
    ground_truth_data, prospector_results
)
print(
    f"{correct_is_top} times the top candidate was the true fixing commit, \n{correct_in_top_3} times the top 3 candidates contained the true fixing commit out of {len(cves_2018)} reports."
)

With Commit Classification Rule: 

In [7]:
with open("../../../" + config.ground_truth, "r") as f1, open(
    "../../../" + config.with_cc, "r"
) as f2:
    ground_truth_data = json.load(f1)  # baseline from project KB
    prospector_results = json.load(f2)  # prospector results without cc

cves_2018 = []
for file in os.listdir("../../../data_sources/reports/"):
    if file.startswith("CVE-2018-"):
        cves_2018.append(file)

correct_is_top, correct_in_top_3 = count_correct_results(
    ground_truth_data, prospector_results
)
print(
    f"{correct_is_top} times the top candidate was the true fixing commit, \n{correct_in_top_3} times the top 3 candidates contained the true fixing commit out of {len(cves_2018)} reports."
)

13 times the top candidate was the true fixing commit, 
15 times the top 3 candidates contained the true fixing commit out of 18 reports.


### First 10 candidates

In how many reports is the fixing commit within the first 10 candidate? This is important to know to see how many commits the LLM rules should be applied to. I should compare the percentages of correct fixing commits for different numbers of candidate commits, eg. 1, 3, 5, and 10. I need to compare using Prospector without the cc rule.

In [6]:
def count_among_first_x_candidates(ground_truth_data, prospector_data):
    count_correct = 0
    count_correct_first_3 = 0
    count_correct_first_5 = 0
    count_correct_first_10 = 0

    for cve_record in ground_truth_data["ground_truth"]:
        cve_id = cve_record["vulnerability_id"]

        prospector_cve_record = prospector_data["vulnerabilities"].get(cve_id, None)
        if prospector_cve_record is not None:
            # print(f"{cve_id} in both baseline and prospector.") # Sanity check

            true_fixing_commits = [
                commit["id"] for fix in cve_record["fixes"] for commit in fix["commits"]
            ]

            if prospector_cve_record["commits"]:
                first_fixing_commit = prospector_cve_record["commits"][0]["commit_hash"]
                first_3_fixing_commits = [
                    commit["commit_hash"]
                    for commit in prospector_cve_record["commits"][:3]
                ]
                first_5_fixing_commits = [
                    commit["commit_hash"]
                    for commit in prospector_cve_record["commits"][:5]
                ]
                first_10_fixing_commits = [
                    commit["commit_hash"]
                    for commit in prospector_cve_record["commits"][:10]
                ]

            if first_fixing_commit in true_fixing_commits:
                count_correct += 1

            if set(first_3_fixing_commits) & set(true_fixing_commits):
                count_correct_first_3 += 1
            if set(first_5_fixing_commits) & set(true_fixing_commits):
                count_correct_first_5 += 1
            if set(first_10_fixing_commits) & set(true_fixing_commits):
                count_correct_first_10 += 1

    return (
        count_correct,
        count_correct_first_3,
        count_correct_first_5,
        count_correct_first_10,
    )

##### 2018 CVE Reports

In [22]:
print("Results without using commit classification:\n")

with open("../../../" + config.ground_truth_2018, "r") as f1, open(
    "../../../" + config.no_cc_2018, "r"
) as f2:
    f1_data = json.load(f1)  # ground truth from Project KB
    f2_data = json.load(f2)  # prospector results (without CC)

oo1, oo3, oo5, oo10 = count_among_first_x_candidates(f1_data, f2_data)
print(
    f"{oo1} times fixing commit among the first candidate commit, \n{oo3} times fixing commit among top 3 candidates ({round(100/len(f2_data['vulnerabilities'])*oo3, 2)}), \n{oo5} times fixing commit among top 5 candidates ({round(100/len(f2_data['vulnerabilities'])*oo5, 2)}), \n{oo10} times fixing commit among top 10 candidates ({round(100/len(f2_data['vulnerabilities'])*oo10, 2)}) out of {len(f2_data['vulnerabilities'])} commits in total."
)

print("\n\nResults with commit classification:\n")

with open("../../../" + config.ground_truth_2018, "r") as f1, open(
    "../../../" + config.cc_2018, "r"
) as f2:
    f1_data = json.load(f1)  # ground truth from Project KB
    f2_data = json.load(f2)  # prospector results (without CC)

oo1, oo3, oo5, oo10 = count_among_first_x_candidates(f1_data, f2_data)
print(
    f"{oo1} times fixing commit among the first candidate commit, \n{oo3} times fixing commit among top 3 candidates ({round(100/len(f2_data['vulnerabilities'])*oo3, 2)}), \n{oo5} times fixing commit among top 5 candidates ({round(100/len(f2_data['vulnerabilities'])*oo5, 2)}), \n{oo10} times fixing commit among top 10 candidates ({round(100/len(f2_data['vulnerabilities'])*oo10, 2)}) out of {len(f2_data['vulnerabilities'])} commits in total."
)

Results without using commit classification:

91 times fixing commit among the first candidate commit, 
112 times fixing commit among top 3 candidates (56.57), 
114 times fixing commit among top 5 candidates (57.58), 
115 times fixing commit among top 10 candidates (58.08) out of 198 commits in total.


Results with commit classification:

